# Integrative Multiomics Network Analysis for Rheumatoid Arthritis

## A Comprehensive Tutorial for Genetic Risk Factor Analysis and Therapeutic Target Discovery

---

**Based on the research methodology from:**
> *Integrative Multiomics Network Analysis of Genetic Risk Factors to Infer Biomarkers and Therapeutic Targets for Rheumatoid Arthritis*  
> Alsaedi, S.B., Mineta, K., Tamura, N., Gao, X., Gojobori, T., Ogasawara, M. (2024)

---

### 🎯 Learning Objectives

After completing this tutorial, you will be able to:

1. **Integrate multiomics data** from genetics, transcriptomics, proteomics, and metabolomics
2. **Construct molecular networks** using protein-protein interactions and functional similarities
3. **Perform pathway enrichment analysis** to identify key biological processes
4. **Identify biomarkers and therapeutic targets** through network analysis
5. **Apply disease mapping** to discover drug repurposing opportunities

### 📊 Dataset Overview

This tutorial uses synthetic datasets that mirror real rheumatoid arthritis research:

- **Genetic Variants**: 199 RA-associated variants across 120 genes
- **Gene Expression**: 200 samples (100 RA patients, 100 controls) across 121 genes
- **Protein Abundance**: Corresponding protein levels for all samples
- **Metabolomics**: 40 metabolites associated with RA pathogenesis
- **Clinical Data**: Disease activity scores, biomarkers, and treatment responses
- **Pathway Annotations**: 7 major biological pathways with 81 gene associations

### 🔬 Methodology Overview

We follow the five-stage computational workflow:

1. **Data Curation and Validation**
2. **Genetic Enrichment Analysis**
3. **Disease Mapping and Drug Target Identification**
4. **Molecular Network Construction**
5. **Network Pathway Analysis and Therapeutic Target Discovery**

## 📦 Installation and Setup

First, let's install and import all required packages:

In [ ]:
# Install required packages (run this cell first in Google Colab)
!pip install pandas numpy matplotlib seaborn plotly networkx scipy scikit-learn umap-learn
!pip install bioservices requests beautifulsoup4

# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import networkx as nx
from scipy import stats
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import umap
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

print("✅ All packages installed and imported successfully!")
print("🔬 Ready to begin rheumatoid arthritis multiomics analysis!")

## 📁 Data Loading and Upload

### For Google Colab Users:
Upload the following CSV files when prompted:
- `ra_genetic_variants.csv`
- `ra_gene_expression.csv`
- `ra_protein_abundance.csv`
- `ra_metabolomics.csv`
- `ra_clinical_data.csv`
- `ra_pathway_annotations.csv`

In [ ]:
# For Google Colab - upload files
try:
    from google.colab import files
    print("📤 Please upload the RA multiomics dataset files...")
    uploaded = files.upload()
    print("✅ Files uploaded successfully!")
except ImportError:
    print("📂 Running locally - files should be in the current directory")

# Load all datasets
print("📊 Loading multiomics datasets...")

try:
    # Load genetic variants
    genetic_df = pd.read_csv('ra_genetic_variants.csv')
    print(f"   ✅ Genetic variants: {genetic_df.shape[0]} variants, {genetic_df.shape[1]} features")
    
    # Load gene expression
    expression_df = pd.read_csv('ra_gene_expression.csv')
    print(f"   ✅ Gene expression: {expression_df.shape[0]} samples, {expression_df.shape[1]} features")
    
    # Load protein abundance
    protein_df = pd.read_csv('ra_protein_abundance.csv')
    print(f"   ✅ Protein abundance: {protein_df.shape[0]} samples, {protein_df.shape[1]} features")
    
    # Load metabolomics
    metabolomics_df = pd.read_csv('ra_metabolomics.csv')
    print(f"   ✅ Metabolomics: {metabolomics_df.shape[0]} samples, {metabolomics_df.shape[1]} features")
    
    # Load clinical data
    clinical_df = pd.read_csv('ra_clinical_data.csv')
    print(f"   ✅ Clinical data: {clinical_df.shape[0]} samples, {clinical_df.shape[1]} features")
    
    # Load pathway annotations
    pathway_df = pd.read_csv('ra_pathway_annotations.csv')
    print(f"   ✅ Pathway annotations: {pathway_df.shape[0]} gene-pathway associations")
    
    print("\n🎉 All datasets loaded successfully!")
    
except FileNotFoundError as e:
    print(f"❌ Error loading files: {e}")
    print("Please ensure all CSV files are uploaded/available in the current directory")

## 🔍 Stage 1: Data Curation and Validation

In this stage, we explore and validate our multiomics datasets to ensure data quality and understand the characteristics of our RA cohort.

In [ ]:
# Data overview and quality assessment
print("🔍 STAGE 1: DATA CURATION AND VALIDATION")
print("=" * 50)

# 1.1 Genetic variants overview
print("\n📊 Genetic Variants Analysis:")
print(f"Total variants: {len(genetic_df)}")
print(f"Unique genes: {genetic_df['gene_symbol'].nunique()}")
print(f"Chromosomes represented: {genetic_df['chromosome'].nunique()}")
print(f"Study types: {genetic_df['study_type'].value_counts().to_dict()}")
print(f"Populations: {genetic_df['population'].value_counts().to_dict()}")

# 1.2 Sample characteristics
print("\n👥 Sample Characteristics:")
print(f"Total samples: {len(clinical_df)}")
print(f"RA patients: {sum(clinical_df['condition'] == 'RA_patient')}")
print(f"Healthy controls: {sum(clinical_df['condition'] == 'healthy_control')}")
print(f"Tissue types: {expression_df['tissue_type'].value_counts().to_dict()}")

# 1.3 Clinical characteristics of RA patients
ra_patients = clinical_df[clinical_df['condition'] == 'RA_patient']
print("\n🏥 RA Patient Clinical Characteristics:")
print(f"Age range: {ra_patients['age'].min():.0f}-{ra_patients['age'].max():.0f} years")
print(f"Sex distribution: {ra_patients['sex'].value_counts().to_dict()}")
print(f"RF positive: {sum(ra_patients['RF_positive'])}/{len(ra_patients)} ({100*sum(ra_patients['RF_positive'])/len(ra_patients):.1f}%)")
print(f"ACPA positive: {sum(ra_patients['ACPA_positive'])}/{len(ra_patients)} ({100*sum(ra_patients['ACPA_positive'])/len(ra_patients):.1f}%)")
print(f"Mean DAS28 score: {ra_patients['DAS28_score'].mean():.2f} ± {ra_patients['DAS28_score'].std():.2f}")

# 1.4 Data quality checks
print("\n✅ Data Quality Assessment:")
datasets = {
    'Genetic': genetic_df,
    'Expression': expression_df,
    'Protein': protein_df,
    'Metabolomics': metabolomics_df,
    'Clinical': clinical_df
}

for name, df in datasets.items():
    missing_pct = (df.isnull().sum().sum() / (df.shape[0] * df.shape[1])) * 100
    print(f"   {name}: {missing_pct:.2f}% missing values")

In [ ]:
# Visualize data characteristics
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=[
        'P-value Distribution', 'Odds Ratio Distribution', 'Allele Frequency Distribution',
        'DAS28 Score by Condition', 'Age Distribution', 'Tissue Type Distribution'
    ],
    specs=[[{"type": "histogram"}, {"type": "histogram"}, {"type": "histogram"}],
           [{"type": "box"}, {"type": "histogram"}, {"type": "pie"}]]
)

# P-value distribution
fig.add_trace(
    go.Histogram(x=-np.log10(genetic_df['p_value']), name='-log10(p-value)', nbinsx=30),
    row=1, col=1
)

# Odds ratio distribution
fig.add_trace(
    go.Histogram(x=genetic_df['odds_ratio'], name='Odds Ratio', nbinsx=30),
    row=1, col=2
)

# Allele frequency distribution
fig.add_trace(
    go.Histogram(x=genetic_df['allele_frequency'], name='Allele Frequency', nbinsx=30),
    row=1, col=3
)

# DAS28 score by condition
for condition in clinical_df['condition'].unique():
    data = clinical_df[clinical_df['condition'] == condition]['DAS28_score']
    fig.add_trace(
        go.Box(y=data, name=condition),
        row=2, col=1
    )

# Age distribution
fig.add_trace(
    go.Histogram(x=clinical_df['age'], name='Age', nbinsx=20),
    row=2, col=2
)

# Tissue type distribution
tissue_counts = expression_df['tissue_type'].value_counts()
fig.add_trace(
    go.Pie(labels=tissue_counts.index, values=tissue_counts.values, name="Tissue Types"),
    row=2, col=3
)

fig.update_layout(
    height=800,
    title_text="Data Characteristics Overview",
    showlegend=False
)

fig.show()

print("📈 Data visualization complete!")
print("✅ Stage 1: Data curation and validation completed successfully!")

## 🧬 Stage 2: Genetic Enrichment Analysis

In this stage, we perform pathway enrichment analysis to identify key biological processes and molecular functions associated with RA genetic risk factors.

In [ ]:
print("🧬 STAGE 2: GENETIC ENRICHMENT ANALYSIS")
print("=" * 50)

# 2.1 Gene-level analysis
print("\n📊 Gene-level Analysis:")

# Aggregate variants by gene
gene_summary = genetic_df.groupby('gene_symbol').agg({
    'variant_id': 'count',
    'p_value': 'min',
    'odds_ratio': 'mean',
    'allele_frequency': 'mean',
    'functional_consequence': lambda x: ', '.join(x.unique()),
    'effect_direction': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 'unknown'
}).rename(columns={'variant_id': 'variant_count'})

gene_summary = gene_summary.sort_values('variant_count', ascending=False)
print(f"Analyzed {len(gene_summary)} genes with genetic variants")
print(f"Top 5 genes by variant count:")
for i, (gene, row) in enumerate(gene_summary.head().iterrows()):
    print(f"  {i+1}. {gene}: {row['variant_count']} variants, min p-value: {row['p_value']:.2e}")

# 2.2 Pathway enrichment analysis
print("\n🛤️ Pathway Enrichment Analysis:")

# Get genes with significant variants (p < 1e-5)
significant_genes = genetic_df[genetic_df['p_value'] < 1e-5]['gene_symbol'].unique()
print(f"Genes with highly significant variants (p < 1e-5): {len(significant_genes)}")

# Calculate pathway enrichment
pathway_enrichment = []
total_genes = len(genetic_df['gene_symbol'].unique())

for pathway in pathway_df['pathway_name'].unique():
    pathway_genes = set(pathway_df[pathway_df['pathway_name'] == pathway]['gene_symbol'])
    significant_in_pathway = len(set(significant_genes) & pathway_genes)
    total_in_pathway = len(pathway_genes)
    
    # Fisher's exact test (simplified)
    enrichment_score = (significant_in_pathway / total_in_pathway) / (len(significant_genes) / total_genes)
    
    pathway_enrichment.append({
        'pathway': pathway,
        'total_genes': total_in_pathway,
        'significant_genes': significant_in_pathway,
        'enrichment_score': enrichment_score,
        'genes_in_pathway': list(pathway_genes & set(significant_genes))
    })

enrichment_df = pd.DataFrame(pathway_enrichment)
enrichment_df = enrichment_df.sort_values('enrichment_score', ascending=False)

print("\nPathway Enrichment Results:")
for _, row in enrichment_df.iterrows():
    print(f"  {row['pathway']}: {row['enrichment_score']:.2f}x enriched ({row['significant_genes']}/{row['total_genes']} genes)")

# 2.3 Functional consequence analysis
print("\n🔬 Functional Consequence Analysis:")
consequence_counts = genetic_df['functional_consequence'].value_counts()
print("Variant functional consequences:")
for consequence, count in consequence_counts.items():
    print(f"  {consequence}: {count} variants ({100*count/len(genetic_df):.1f}%)")

In [ ]:
# Visualize enrichment analysis results
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        'Pathway Enrichment Scores', 'Functional Consequences',
        'Gene Variant Counts', 'P-value vs Odds Ratio'
    ],
    specs=[[{"type": "bar"}, {"type": "pie"}],
           [{"type": "bar"}, {"type": "scatter"}]]
)

# Pathway enrichment scores
fig.add_trace(
    go.Bar(
        x=enrichment_df['pathway'],
        y=enrichment_df['enrichment_score'],
        name='Enrichment Score',
        marker_color='lightblue'
    ),
    row=1, col=1
)

# Functional consequences pie chart
fig.add_trace(
    go.Pie(
        labels=consequence_counts.index,
        values=consequence_counts.values,
        name="Functional Consequences"
    ),
    row=1, col=2
)

# Top genes by variant count
top_genes = gene_summary.head(15)
fig.add_trace(
    go.Bar(
        x=top_genes.index,
        y=top_genes['variant_count'],
        name='Variant Count',
        marker_color='lightcoral'
    ),
    row=2, col=1
)

# P-value vs Odds Ratio scatter plot
fig.add_trace(
    go.Scatter(
        x=-np.log10(genetic_df['p_value']),
        y=genetic_df['odds_ratio'],
        mode='markers',
        name='Variants',
        text=genetic_df['gene_symbol'],
        marker=dict(size=6, opacity=0.6)
    ),
    row=2, col=2
)

# Update layout
fig.update_layout(
    height=800,
    title_text="Genetic Enrichment Analysis Results",
    showlegend=False
)

# Update x-axis labels for pathway plot
fig.update_xaxes(tickangle=45, row=1, col=1)
fig.update_xaxes(tickangle=45, row=2, col=1)
fig.update_xaxes(title_text="-log10(p-value)", row=2, col=2)
fig.update_yaxes(title_text="Odds Ratio", row=2, col=2)

fig.show()

print("📊 Enrichment analysis visualization complete!")
print("✅ Stage 2: Genetic enrichment analysis completed successfully!")

## 🎯 Stage 3: Disease Mapping and Drug Target Identification

In this stage, we integrate clinical data with genetic and molecular data to identify potential biomarkers and therapeutic targets.

In [ ]:
print("🎯 STAGE 3: DISEASE MAPPING AND DRUG TARGET IDENTIFICATION")
print("=" * 60)

# 3.1 Biomarker identification from expression data
print("\n🔬 Biomarker Identification:")

# Get expression columns (exclude metadata)
expression_cols = [col for col in expression_df.columns if col.endswith('_expression')]
gene_names = [col.replace('_expression', '') for col in expression_cols]

# Perform differential expression analysis
ra_samples = expression_df[expression_df['condition'] == 'RA_patient']
control_samples = expression_df[expression_df['condition'] == 'healthy_control']

biomarkers = []
for col, gene in zip(expression_cols, gene_names):
    ra_expr = ra_samples[col]
    control_expr = control_samples[col]
    
    # T-test for differential expression
    t_stat, p_val = stats.ttest_ind(ra_expr, control_expr)
    fold_change = ra_expr.mean() / control_expr.mean()
    log2_fc = np.log2(fold_change)
    
    biomarkers.append({
        'gene': gene,
        'log2_fold_change': log2_fc,
        'p_value': p_val,
        'ra_mean': ra_expr.mean(),
        'control_mean': control_expr.mean(),
        'significant': p_val < 0.05 and abs(log2_fc) > 0.5
    })

biomarkers_df = pd.DataFrame(biomarkers)
biomarkers_df = biomarkers_df.sort_values('p_value')

significant_biomarkers = biomarkers_df[biomarkers_df['significant']]
print(f"Identified {len(significant_biomarkers)} significant biomarkers (p < 0.05, |log2FC| > 0.5)")

print("\nTop 10 Biomarkers:")
for _, row in significant_biomarkers.head(10).iterrows():
    direction = "↑" if row['log2_fold_change'] > 0 else "↓"
    print(f"  {row['gene']}: {direction} {row['log2_fold_change']:.2f} log2FC, p={row['p_value']:.2e}")

# 3.2 Therapeutic target scoring
print("\n🎯 Therapeutic Target Scoring:")

# Combine genetic and expression evidence
target_scores = []
for gene in gene_names:
    # Genetic evidence
    genetic_evidence = 0
    if gene in genetic_df['gene_symbol'].values:
        gene_variants = genetic_df[genetic_df['gene_symbol'] == gene]
        min_p = gene_variants['p_value'].min()
        genetic_evidence = -np.log10(min_p) if min_p > 0 else 10
    
    # Expression evidence
    expression_evidence = 0
    if gene in biomarkers_df['gene'].values:
        biomarker_data = biomarkers_df[biomarkers_df['gene'] == gene].iloc[0]
        expression_evidence = -np.log10(biomarker_data['p_value']) if biomarker_data['p_value'] > 0 else 10
    
    # Pathway evidence
    pathway_evidence = 0
    if gene in pathway_df['gene_symbol'].values:
        gene_pathways = pathway_df[pathway_df['gene_symbol'] == gene]
        pathway_evidence = len(gene_pathways)  # Number of pathways
    
    # Combined score
    combined_score = (genetic_evidence * 0.4) + (expression_evidence * 0.4) + (pathway_evidence * 0.2)
    
    target_scores.append({
        'gene': gene,
        'genetic_evidence': genetic_evidence,
        'expression_evidence': expression_evidence,
        'pathway_evidence': pathway_evidence,
        'combined_score': combined_score
    })

targets_df = pd.DataFrame(target_scores)
targets_df = targets_df.sort_values('combined_score', ascending=False)

top_targets = targets_df.head(15)
print(f"\nTop 15 Therapeutic Targets:")
for i, (_, row) in enumerate(top_targets.iterrows()):
    print(f"  {i+1:2d}. {row['gene']:10s}: Score={row['combined_score']:.2f} "
          f"(G:{row['genetic_evidence']:.1f}, E:{row['expression_evidence']:.1f}, P:{row['pathway_evidence']:.0f})")

# 3.3 Clinical correlation analysis
print("\n🏥 Clinical Correlation Analysis:")

# Correlate top biomarkers with clinical scores
ra_clinical = clinical_df[clinical_df['condition'] == 'RA_patient'].copy()
ra_expression = expression_df[expression_df['condition'] == 'RA_patient'].copy()

# Merge clinical and expression data
merged_data = pd.merge(ra_clinical, ra_expression, on='sample_id')

clinical_correlations = []
clinical_measures = ['DAS28_score', 'HAQ_score', 'ESR_mm_hr', 'CRP_mg_L']

for gene in significant_biomarkers.head(10)['gene']:
    expr_col = f'{gene}_expression'
    if expr_col in merged_data.columns:
        for measure in clinical_measures:
            corr, p_val = stats.pearsonr(merged_data[expr_col], merged_data[measure])
            clinical_correlations.append({
                'gene': gene,
                'clinical_measure': measure,
                'correlation': corr,
                'p_value': p_val
            })

correlations_df = pd.DataFrame(clinical_correlations)
significant_correlations = correlations_df[correlations_df['p_value'] < 0.05]

print(f"Found {len(significant_correlations)} significant gene-clinical correlations (p < 0.05)")
if len(significant_correlations) > 0:
    print("\nTop Clinical Correlations:")
    for _, row in significant_correlations.head(10).iterrows():
        print(f"  {row['gene']} - {row['clinical_measure']}: r={row['correlation']:.3f}, p={row['p_value']:.3f}")

In [ ]:
# Visualize disease mapping results
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        'Volcano Plot (Differential Expression)', 'Therapeutic Target Scores',
        'Clinical Correlations Heatmap', 'Biomarker Expression Comparison'
    ],
    specs=[[{"type": "scatter"}, {"type": "bar"}],
           [{"type": "heatmap"}, {"type": "box"}]]
)

# Volcano plot
colors = ['red' if sig else 'gray' for sig in biomarkers_df['significant']]
fig.add_trace(
    go.Scatter(
        x=biomarkers_df['log2_fold_change'],
        y=-np.log10(biomarkers_df['p_value']),
        mode='markers',
        marker=dict(color=colors, size=6),
        text=biomarkers_df['gene'],
        name='Genes'
    ),
    row=1, col=1
)

# Therapeutic target scores
fig.add_trace(
    go.Bar(
        x=top_targets['gene'],
        y=top_targets['combined_score'],
        name='Target Score',
        marker_color='lightgreen'
    ),
    row=1, col=2
)

# Clinical correlations heatmap (if we have significant correlations)
if len(significant_correlations) > 0:
    # Create correlation matrix
    corr_matrix = significant_correlations.pivot(index='gene', columns='clinical_measure', values='correlation')
    corr_matrix = corr_matrix.fillna(0)
    
    fig.add_trace(
        go.Heatmap(
            z=corr_matrix.values,
            x=corr_matrix.columns,
            y=corr_matrix.index,
            colorscale='RdBu',
            zmid=0,
            name='Correlation'
        ),
        row=2, col=1
    )

# Biomarker expression comparison (top 3 biomarkers)
top_biomarker_genes = significant_biomarkers.head(3)['gene'].tolist()
for gene in top_biomarker_genes:
    expr_col = f'{gene}_expression'
    if expr_col in expression_df.columns:
        for condition in ['RA_patient', 'healthy_control']:
            data = expression_df[expression_df['condition'] == condition][expr_col]
            fig.add_trace(
                go.Box(
                    y=data,
                    name=f'{gene}_{condition}',
                    boxpoints='outliers'
                ),
                row=2, col=2
            )

# Update layout
fig.update_layout(
    height=800,
    title_text="Disease Mapping and Target Identification Results",
    showlegend=False
)

# Update axis labels
fig.update_xaxes(title_text="log2(Fold Change)", row=1, col=1)
fig.update_yaxes(title_text="-log10(p-value)", row=1, col=1)
fig.update_xaxes(tickangle=45, row=1, col=2)
fig.update_xaxes(tickangle=45, row=2, col=2)

fig.show()

print("📊 Disease mapping visualization complete!")
print("✅ Stage 3: Disease mapping and drug target identification completed successfully!")

## 🕸️ Stage 4: Molecular Network Construction

In this stage, we construct protein-protein interaction networks and analyze network topology to identify key regulatory nodes and functional modules.

In [ ]:
print("🕸️ STAGE 4: MOLECULAR NETWORK CONSTRUCTION")
print("=" * 50)

# 4.1 Construct protein-protein interaction network
print("\n🔗 Constructing Protein-Protein Interaction Network:")

# Use top therapeutic targets for network construction
network_genes = top_targets.head(30)['gene'].tolist()
print(f"Building network with {len(network_genes)} genes")

# Create network based on functional similarities and pathway associations
def create_ra_network(genes, pathway_df, expression_df):
    G = nx.Graph()
    
    # Add nodes
    for gene in genes:
        G.add_node(gene)
    
    # Add edges based on pathway co-membership and expression correlation
    for i, gene1 in enumerate(genes):
        for j, gene2 in enumerate(genes[i+1:], i+1):
            # Pathway-based connection
            gene1_pathways = set(pathway_df[pathway_df['gene_symbol'] == gene1]['pathway_name'])
            gene2_pathways = set(pathway_df[pathway_df['gene_symbol'] == gene2]['pathway_name'])
            shared_pathways = len(gene1_pathways & gene2_pathways)
            
            # Expression correlation
            expr1_col = f'{gene1}_expression'
            expr2_col = f'{gene2}_expression'
            
            correlation = 0
            if expr1_col in expression_df.columns and expr2_col in expression_df.columns:
                corr, p_val = stats.pearsonr(expression_df[expr1_col], expression_df[expr2_col])
                if p_val < 0.05:
                    correlation = abs(corr)
            
            # Create edge if there's evidence for interaction
            if shared_pathways > 0 or correlation > 0.3:
                weight = (shared_pathways * 0.5) + (correlation * 0.5)
                G.add_edge(gene1, gene2, weight=weight, 
                          shared_pathways=shared_pathways, 
                          correlation=correlation)
    
    return G

# Build the network
ppi_network = create_ra_network(network_genes, pathway_df, expression_df)

print(f"Network constructed with:")
print(f"  Nodes: {ppi_network.number_of_nodes()}")
print(f"  Edges: {ppi_network.number_of_edges()}")
print(f"  Density: {nx.density(ppi_network):.3f}")

# 4.2 Network topology analysis
print("\n📊 Network Topology Analysis:")

if ppi_network.number_of_edges() > 0:
    # Calculate centrality measures
    degree_centrality = nx.degree_centrality(ppi_network)
    betweenness_centrality = nx.betweenness_centrality(ppi_network)
    closeness_centrality = nx.closeness_centrality(ppi_network)
    eigenvector_centrality = nx.eigenvector_centrality(ppi_network, max_iter=1000)
    
    # Create centrality dataframe
    centrality_df = pd.DataFrame({
        'gene': list(degree_centrality.keys()),
        'degree_centrality': list(degree_centrality.values()),
        'betweenness_centrality': list(betweenness_centrality.values()),
        'closeness_centrality': list(closeness_centrality.values()),
        'eigenvector_centrality': list(eigenvector_centrality.values())
    })
    
    # Calculate hub score (average of normalized centralities)
    for col in ['degree_centrality', 'betweenness_centrality', 'closeness_centrality', 'eigenvector_centrality']:
        centrality_df[f'{col}_norm'] = (centrality_df[col] - centrality_df[col].min()) / (centrality_df[col].max() - centrality_df[col].min())
    
    centrality_df['hub_score'] = centrality_df[['degree_centrality_norm', 'betweenness_centrality_norm', 
                                              'closeness_centrality_norm', 'eigenvector_centrality_norm']].mean(axis=1)
    
    centrality_df = centrality_df.sort_values('hub_score', ascending=False)
    
    print("Top 10 Hub Genes:")
    for i, (_, row) in enumerate(centrality_df.head(10).iterrows()):
        print(f"  {i+1:2d}. {row['gene']:10s}: Hub Score={row['hub_score']:.3f}, "
              f"Degree={row['degree_centrality']:.3f}, Betweenness={row['betweenness_centrality']:.3f}")
    
    # 4.3 Community detection
    print("\n🏘️ Community Detection:")
    
    if ppi_network.number_of_edges() > 0:
        # Use Louvain method for community detection
        try:
            communities = nx.community.louvain_communities(ppi_network)
            print(f"Detected {len(communities)} communities")
            
            community_info = []
            for i, community in enumerate(communities):
                community_genes = list(community)
                # Find dominant pathway for this community
                community_pathways = []
                for gene in community_genes:
                    gene_pathways = pathway_df[pathway_df['gene_symbol'] == gene]['pathway_name'].tolist()
                    community_pathways.extend(gene_pathways)
                
                if community_pathways:
                    dominant_pathway = max(set(community_pathways), key=community_pathways.count)
                else:
                    dominant_pathway = "Unknown"
                
                community_info.append({
                    'community_id': i,
                    'size': len(community_genes),
                    'genes': community_genes,
                    'dominant_pathway': dominant_pathway
                })
                
                print(f"  Community {i+1}: {len(community_genes)} genes, dominant pathway: {dominant_pathway}")
                print(f"    Genes: {', '.join(community_genes[:5])}{'...' if len(community_genes) > 5 else ''}")
        
        except:
            print("Community detection failed - using simple clustering")
            communities = [set(network_genes)]
    
else:
    print("Network has no edges - skipping topology analysis")
    centrality_df = pd.DataFrame({'gene': network_genes, 'hub_score': [0]*len(network_genes)})
    communities = [set(network_genes)]

In [ ]:
# Visualize network and topology
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        'Protein-Protein Interaction Network', 'Hub Gene Centrality Scores',
        'Degree Distribution', 'Community Sizes'
    ],
    specs=[[{"type": "scatter"}, {"type": "bar"}],
           [{"type": "histogram"}, {"type": "bar"}]]
)

# Network visualization
if ppi_network.number_of_edges() > 0:
    # Use spring layout for network positioning
    pos = nx.spring_layout(ppi_network, k=1, iterations=50)
    
    # Add edges
    edge_x = []
    edge_y = []
    for edge in ppi_network.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
    
    fig.add_trace(
        go.Scatter(
            x=edge_x, y=edge_y,
            line=dict(width=0.5, color='#888'),
            hoverinfo='none',
            mode='lines',
            name='Edges'
        ),
        row=1, col=1
    )
    
    # Add nodes
    node_x = [pos[node][0] for node in ppi_network.nodes()]
    node_y = [pos[node][1] for node in ppi_network.nodes()]
    node_text = list(ppi_network.nodes())
    
    # Color nodes by hub score
    node_colors = [centrality_df[centrality_df['gene'] == node]['hub_score'].iloc[0] 
                   if node in centrality_df['gene'].values else 0 
                   for node in ppi_network.nodes()]
    
    fig.add_trace(
        go.Scatter(
            x=node_x, y=node_y,
            mode='markers+text',
            marker=dict(
                size=10,
                color=node_colors,
                colorscale='Viridis',
                showscale=True,
                colorbar=dict(title="Hub Score")
            ),
            text=node_text,
            textposition="middle center",
            textfont=dict(size=8),
            hoverinfo='text',
            name='Genes'
        ),
        row=1, col=1
    )

# Hub gene centrality scores
top_hubs = centrality_df.head(15)
fig.add_trace(
    go.Bar(
        x=top_hubs['gene'],
        y=top_hubs['hub_score'],
        name='Hub Score',
        marker_color='lightblue'
    ),
    row=1, col=2
)

# Degree distribution
if ppi_network.number_of_edges() > 0:
    degrees = [ppi_network.degree(node) for node in ppi_network.nodes()]
    fig.add_trace(
        go.Histogram(
            x=degrees,
            nbinsx=10,
            name='Degree Distribution'
        ),
        row=2, col=1
    )

# Community sizes
if 'community_info' in locals():
    community_sizes = [info['size'] for info in community_info]
    community_labels = [f"Community {i+1}" for i in range(len(community_sizes))]
    
    fig.add_trace(
        go.Bar(
            x=community_labels,
            y=community_sizes,
            name='Community Size',
            marker_color='lightcoral'
        ),
        row=2, col=2
    )

# Update layout
fig.update_layout(
    height=800,
    title_text="Molecular Network Analysis Results",
    showlegend=False
)

# Update axis labels
fig.update_xaxes(tickangle=45, row=1, col=2)
fig.update_xaxes(title_text="Degree", row=2, col=1)
fig.update_yaxes(title_text="Count", row=2, col=1)
fig.update_xaxes(tickangle=45, row=2, col=2)

# Remove axes for network plot
fig.update_xaxes(showticklabels=False, row=1, col=1)
fig.update_yaxes(showticklabels=False, row=1, col=1)

fig.show()

print("🕸️ Network visualization complete!")
print("✅ Stage 4: Molecular network construction completed successfully!")

## 🎯 Stage 5: Network Pathway Analysis and Therapeutic Target Discovery

In this final stage, we integrate all multiomics data to identify the most promising therapeutic targets and potential drug repurposing opportunities.

In [ ]:
print("🎯 STAGE 5: NETWORK PATHWAY ANALYSIS AND THERAPEUTIC TARGET DISCOVERY")
print("=" * 70)

# 5.1 Integrate multiomics evidence for final target ranking
print("\n🔬 Integrative Target Ranking:")

# Combine all evidence types
final_targets = []

for gene in network_genes:
    # Genetic evidence
    genetic_score = 0
    if gene in genetic_df['gene_symbol'].values:
        gene_variants = genetic_df[genetic_df['gene_symbol'] == gene]
        min_p = gene_variants['p_value'].min()
        genetic_score = min(-np.log10(min_p), 10) if min_p > 0 else 10
    
    # Expression evidence
    expression_score = 0
    if gene in biomarkers_df['gene'].values:
        biomarker_data = biomarkers_df[biomarkers_df['gene'] == gene].iloc[0]
        if biomarker_data['significant']:
            expression_score = min(-np.log10(biomarker_data['p_value']), 10)
    
    # Network evidence (hub score)
    network_score = 0
    if gene in centrality_df['gene'].values:
        network_score = centrality_df[centrality_df['gene'] == gene]['hub_score'].iloc[0] * 10
    
    # Pathway evidence
    pathway_score = 0
    if gene in pathway_df['gene_symbol'].values:
        gene_pathways = pathway_df[pathway_df['gene_symbol'] == gene]
        pathway_score = len(gene_pathways) * 2  # Weight by number of pathways
    
    # Clinical correlation evidence
    clinical_score = 0
    if gene in correlations_df['gene'].values:
        gene_correlations = correlations_df[correlations_df['gene'] == gene]
        significant_corr = gene_correlations[gene_correlations['p_value'] < 0.05]
        if len(significant_corr) > 0:
            clinical_score = len(significant_corr) * 2
    
    # Protein evidence (if available)
    protein_score = 0
    protein_col = f'{gene}_protein'
    if protein_col in protein_df.columns:
        ra_protein = protein_df[protein_df['condition'] == 'RA_patient'][protein_col]
        control_protein = protein_df[protein_df['condition'] == 'healthy_control'][protein_col]
        if len(ra_protein) > 0 and len(control_protein) > 0:
            t_stat, p_val = stats.ttest_ind(ra_protein, control_protein)
            if p_val < 0.05:
                protein_score = min(-np.log10(p_val), 10)
    
    # Combined score with weights
    weights = {
        'genetic': 0.25,
        'expression': 0.20,
        'network': 0.20,
        'pathway': 0.15,
        'clinical': 0.10,
        'protein': 0.10
    }
    
    combined_score = (
        genetic_score * weights['genetic'] +
        expression_score * weights['expression'] +
        network_score * weights['network'] +
        pathway_score * weights['pathway'] +
        clinical_score * weights['clinical'] +
        protein_score * weights['protein']
    )
    
    final_targets.append({
        'gene': gene,
        'genetic_score': genetic_score,
        'expression_score': expression_score,
        'network_score': network_score,
        'pathway_score': pathway_score,
        'clinical_score': clinical_score,
        'protein_score': protein_score,
        'combined_score': combined_score
    })

final_targets_df = pd.DataFrame(final_targets)
final_targets_df = final_targets_df.sort_values('combined_score', ascending=False)

print("\n🏆 TOP 15 THERAPEUTIC TARGETS:")
print("=" * 50)
for i, (_, row) in enumerate(final_targets_df.head(15).iterrows()):
    print(f"{i+1:2d}. {row['gene']:12s} | Score: {row['combined_score']:5.2f} | "
          f"G:{row['genetic_score']:4.1f} E:{row['expression_score']:4.1f} "
          f"N:{row['network_score']:4.1f} P:{row['pathway_score']:4.1f} "
          f"C:{row['clinical_score']:4.1f} Pr:{row['protein_score']:4.1f}")

# 5.2 Drug repurposing analysis
print("\n💊 Drug Repurposing Analysis:")

# Define known drug targets and their associated diseases
drug_targets = {
    'TNF': ['Adalimumab', 'Infliximab', 'Etanercept', 'Certolizumab'],
    'IL6': ['Tocilizumab', 'Sarilumab'],
    'IL1B': ['Anakinra', 'Canakinumab'],
    'CD20': ['Rituximab'],
    'CTLA4': ['Abatacept'],
    'IL17A': ['Secukinumab', 'Ixekizumab'],
    'IL23R': ['Ustekinumab', 'Guselkumab'],
    'NFKB1': ['Sulfasalazine', 'Methotrexate'],
    'STAT1': ['JAK inhibitors'],
    'STAT3': ['JAK inhibitors']
}

repurposing_candidates = []
top_targets_list = final_targets_df.head(20)['gene'].tolist()

for gene in top_targets_list:
    if gene in drug_targets:
        target_score = final_targets_df[final_targets_df['gene'] == gene]['combined_score'].iloc[0]
        drugs = drug_targets[gene]
        
        for drug in drugs:
            repurposing_candidates.append({
                'target_gene': gene,
                'drug_name': drug,
                'target_score': target_score,
                'repurposing_potential': 'High' if target_score > 3 else 'Medium' if target_score > 2 else 'Low'
            })

if repurposing_candidates:
    repurposing_df = pd.DataFrame(repurposing_candidates)
    repurposing_df = repurposing_df.sort_values('target_score', ascending=False)
    
    print("\n💊 Drug Repurposing Candidates:")
    for _, row in repurposing_df.head(10).iterrows():
        print(f"  {row['drug_name']:15s} → {row['target_gene']:10s} | "
              f"Score: {row['target_score']:5.2f} | Potential: {row['repurposing_potential']}")

# 5.3 Pathway-based therapeutic insights
print("\n🛤️ Pathway-Based Therapeutic Insights:")

pathway_targets = {}
for _, row in final_targets_df.head(20).iterrows():
    gene = row['gene']
    score = row['combined_score']
    
    gene_pathways = pathway_df[pathway_df['gene_symbol'] == gene]['pathway_name'].tolist()
    for pathway in gene_pathways:
        if pathway not in pathway_targets:
            pathway_targets[pathway] = []
        pathway_targets[pathway].append((gene, score))

# Sort pathways by average target score
pathway_scores = []
for pathway, targets in pathway_targets.items():
    avg_score = np.mean([score for _, score in targets])
    pathway_scores.append((pathway, avg_score, len(targets), targets))

pathway_scores.sort(key=lambda x: x[1], reverse=True)

print("\nTop Therapeutic Pathways:")
for pathway, avg_score, n_targets, targets in pathway_scores[:7]:
    top_genes = [gene for gene, _ in sorted(targets, key=lambda x: x[1], reverse=True)[:3]]
    print(f"  {pathway:25s} | Avg Score: {avg_score:5.2f} | Targets: {n_targets} | Top genes: {', '.join(top_genes)}")

print("\n✅ Stage 5: Network pathway analysis and therapeutic target discovery completed successfully!")

In [ ]:
# Final comprehensive visualization
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=[
        'Top Therapeutic Targets', 'Evidence Type Contributions',
        'Drug Repurposing Potential', 'Pathway Target Scores',
        'Target Score Distribution', 'Multiomics Integration Summary'
    ],
    specs=[[{"type": "bar"}, {"type": "bar"}, {"type": "scatter"}],
           [{"type": "bar"}, {"type": "histogram"}, {"type": "bar"}]]
)

# Top therapeutic targets
top_15_targets = final_targets_df.head(15)
fig.add_trace(
    go.Bar(
        x=top_15_targets['gene'],
        y=top_15_targets['combined_score'],
        name='Combined Score',
        marker_color='gold'
    ),
    row=1, col=1
)

# Evidence type contributions (stacked bar for top 10 targets)
top_10_targets = final_targets_df.head(10)
evidence_types = ['genetic_score', 'expression_score', 'network_score', 'pathway_score', 'clinical_score', 'protein_score']
colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown']

for i, evidence_type in enumerate(evidence_types):
    fig.add_trace(
        go.Bar(
            x=top_10_targets['gene'],
            y=top_10_targets[evidence_type],
            name=evidence_type.replace('_score', '').title(),
            marker_color=colors[i]
        ),
        row=1, col=2
    )

# Drug repurposing potential (if we have candidates)
if 'repurposing_df' in locals() and len(repurposing_df) > 0:
    fig.add_trace(
        go.Scatter(
            x=repurposing_df['target_score'],
            y=range(len(repurposing_df)),
            mode='markers+text',
            text=repurposing_df['drug_name'],
            textposition="middle right",
            marker=dict(
                size=10,
                color=repurposing_df['target_score'],
                colorscale='Viridis'
            ),
            name='Repurposing Candidates'
        ),
        row=1, col=3
    )

# Pathway target scores
if pathway_scores:
    pathway_names = [p[0] for p in pathway_scores[:7]]
    pathway_avg_scores = [p[1] for p in pathway_scores[:7]]
    
    fig.add_trace(
        go.Bar(
            x=pathway_names,
            y=pathway_avg_scores,
            name='Pathway Score',
            marker_color='lightgreen'
        ),
        row=2, col=1
    )

# Target score distribution
fig.add_trace(
    go.Histogram(
        x=final_targets_df['combined_score'],
        nbinsx=20,
        name='Score Distribution'
    ),
    row=2, col=2
)

# Multiomics integration summary (data type coverage)
data_coverage = {
    'Genetic Variants': len(genetic_df),
    'Gene Expression': len(expression_df),
    'Protein Abundance': len(protein_df),
    'Metabolomics': len(metabolomics_df),
    'Clinical Data': len(clinical_df),
    'Pathway Annotations': len(pathway_df)
}

fig.add_trace(
    go.Bar(
        x=list(data_coverage.keys()),
        y=list(data_coverage.values()),
        name='Data Coverage',
        marker_color='lightcoral'
    ),
    row=2, col=3
)

# Update layout
fig.update_layout(
    height=800,
    title_text="Comprehensive Therapeutic Target Discovery Results",
    showlegend=False
)

# Update axis labels and formatting
fig.update_xaxes(tickangle=45, row=1, col=1)
fig.update_xaxes(tickangle=45, row=1, col=2)
fig.update_xaxes(title_text="Target Score", row=1, col=3)
fig.update_yaxes(title_text="Drug Rank", row=1, col=3)
fig.update_xaxes(tickangle=45, row=2, col=1)
fig.update_xaxes(title_text="Combined Score", row=2, col=2)
fig.update_yaxes(title_text="Frequency", row=2, col=2)
fig.update_xaxes(tickangle=45, row=2, col=3)

fig.show()

print("📊 Final comprehensive visualization complete!")
print("\n🎉 RHEUMATOID ARTHRITIS MULTIOMICS ANALYSIS COMPLETED SUCCESSFULLY!")
print("=" * 70)

## 📋 Summary and Key Findings

### 🎯 Top Therapeutic Targets Identified

Our integrative multiomics analysis has identified several high-priority therapeutic targets for rheumatoid arthritis based on multiple lines of evidence:

1. **Genetic Evidence**: Variants with strong association signals (p < 1e-5)
2. **Expression Evidence**: Significantly dysregulated genes in RA patients
3. **Network Evidence**: Hub genes with high centrality in protein interaction networks
4. **Pathway Evidence**: Genes involved in multiple RA-relevant biological pathways
5. **Clinical Evidence**: Genes correlated with disease activity measures
6. **Protein Evidence**: Differentially abundant proteins in RA vs. controls

### 💊 Drug Repurposing Opportunities

The analysis identified several existing drugs that could potentially be repurposed for RA treatment based on their targets appearing in our high-priority gene list.

### 🛤️ Key Biological Pathways

The most therapeutically relevant pathways identified include:
- **Immune Response**: HLA genes, STAT signaling, IRF transcription factors
- **Inflammatory Response**: TNF, interleukins, chemokines
- **T Cell Activation**: CD molecules, FOXP3, transcription factors
- **B Cell Function**: CD19/20/22, BAFF/APRIL signaling
- **Bone/Cartilage Metabolism**: RANKL/OPG, matrix metalloproteinases
- **Complement/Coagulation**: Complement cascade, coagulation factors

### 🔬 Methodological Insights

This tutorial demonstrates the power of integrative multiomics analysis for:
1. **Comprehensive target identification** using multiple data types
2. **Network-based prioritization** to identify key regulatory nodes
3. **Clinical relevance assessment** through correlation with disease measures
4. **Drug repurposing discovery** by mapping targets to existing therapeutics

### 🚀 Next Steps

To extend this analysis, consider:
1. **Validation studies** with independent datasets
2. **Functional experiments** to confirm target relevance
3. **Drug screening** against identified targets
4. **Clinical trials** for promising repurposing candidates
5. **Personalized medicine** approaches based on patient-specific profiles

---

**🎓 Congratulations!** You have successfully completed the rheumatoid arthritis multiomics network analysis tutorial. You now have the skills to apply these methods to other complex diseases and datasets.

## 📚 References and Further Reading

### Primary Reference
- Alsaedi, S.B., Mineta, K., Tamura, N., Gao, X., Gojobori, T., Ogasawara, M. (2024). Integrative Multiomics Network Analysis of Genetic Risk Factors to Infer Biomarkers and Therapeutic Targets for Rheumatoid Arthritis. *Research Square*. DOI: 10.21203/rs.3.rs-3607429/v1

### Key Resources
- **STRING Database**: https://string-db.org/ - Protein-protein interactions
- **Reactome**: https://reactome.org/ - Pathway analysis
- **KEGG**: https://www.genome.jp/kegg/ - Biological pathways
- **Gene Ontology**: http://geneontology.org/ - Functional annotations
- **DrugBank**: https://go.drugbank.com/ - Drug-target interactions

### Additional Reading
- Network medicine approaches in autoimmune diseases
- Multiomics integration methods and best practices
- Drug repurposing strategies in rheumatology
- Systems biology of rheumatoid arthritis

---

**📧 Questions or Issues?**
- Check the troubleshooting guide in the repository
- Review the methodology documentation
- Consult the original research paper for detailed methods

**⭐ Found this tutorial helpful?**
- Star the GitHub repository
- Share with colleagues in computational biology
- Contribute improvements or extensions